##Женщины

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Чтение данных из Excel-файла
df = pd.read_excel('/content/Ассортимент брендов_ЖО.xlsx')

# Поиск минимального и максимального значений в столбце "Количество отзывов"
min_reviews = df['Количество отзывов'].min()
max_reviews = df['Количество отзывов'].max()

# Печать результатов
print(f"Минимальное количество отзывов: {min_reviews}")
print(f"Максимальное количество отзывов: {max_reviews}")

Минимальное количество отзывов: 2
Максимальное количество отзывов: 354


Первичная обработка данных

In [2]:
# Удаление строк, где количество отзывов меньше 20
df = df[df['Количество отзывов'] >= 20]
# Перевод значений в столбце "Наименование товара" в нижний регистр и удаление пунктуации
df['Наименование'] = df['Наименование'].str.lower().str.replace('[^\w\s]', '')
# Удаление символа рубля из столбца "Цена"
df['Цена'] = df['Цена'].str.replace('₽', '')
# Удаление пробелов внутри чисел в столбце "Цена"
df['Цена'] = df['Цена'].str.replace('\s+', '', regex=True)
# Удаление всех чисел после точки в столбце "Цена"
df['Цена'] = df['Цена'].str.replace('\.\d+', '', regex=True)
# Сохранение изменений в новый Excel-файл
df.to_excel('Данные_обработанные_ЖО.xlsx', index=False)  # index=False для удаления индексов строк


Добавление признаков

In [3]:
# Функция для поиска цветов в строке
def find_colors(row):
    # Чтение списка цветов из файла
    colors = pd.read_csv('/content/Цвета.txt', header=None)[0].tolist()
    return ', '.join(color for color in colors if color in row['Наименование'])
# Применяем функцию к каждой строке и сохраняем результаты в новый столбец "Цвет"
df['Цвет'] = df.apply(find_colors, axis=1)

# Подсчет общего количества NaN в столбце "Цвет"
num_missing_colors = df['Цвет'].isnull().sum()
# Печать общего количества NaN в столбце "Цвет"


def find_category(row):
  # Чтение списка цветов из файла
    category = pd.read_csv('/content/Категории.txt', header=None)[0].tolist()
    return ', '.join(category for category in category if category in row['Наименование'])
# Применяем функцию к каждой строке и сохраняем результаты в новый столбец "Цвет"
df['Категория'] = df.apply(find_category, axis=1)

def find_material(row):
  # Чтение списка цветов из файла
    material = pd.read_csv('/content/Материал.txt', header=None)[0].tolist()
    return ', '.join(material for material in material if material in row['Наименование'])
# Применяем функцию к каждой строке и сохраняем результаты в новый столбец "Цвет"
df['Материал'] = df.apply(find_material, axis=1)

# Преобразование пустых строк в NaN
df['Цвет'] = df['Цвет'].replace('', np.nan)
df['Категория'] = df['Категория'].replace('', np.nan)
df['Материал'] = df['Материал'].replace('', np.nan)

# Сохранение изменений в новый Excel-файл
df.to_excel('Данные_обработанные_ЖО.xlsx', index=False)  # index=False для удаления индексов строк

# Подсчет общего количества NaN в столбце "Цвет"
num_missing_colors = df['Цвет'].isnull().sum()
num_missing_category = df['Категория'].isnull().sum()
num_missing_material = df['Материал'].isnull().sum()

print(f"Количество NaN в столбце 'Цвет':{num_missing_colors}")
print(f"Количество NaN в столбце 'Категория':{num_missing_category}")
print(f"Количество NaN в столбце 'Материал':{num_missing_material}")

Количество NaN в столбце 'Цвет':298
Количество NaN в столбце 'Категория':539
Количество NaN в столбце 'Материал':1870


In [4]:
# Удаление строк с пропусками в указанных столбцах
df = df.dropna(subset=['Материал', 'Категория', 'Цвет', 'Цена'])

# Сохранение изменений в новый Excel-файл
df.to_excel('Данные_обработанные_ЖО.xlsx', index=False)  # index=False для удаления индексов строк

In [5]:
df = pd.read_excel('/content/Данные_обработанные_ЖО.xlsx')
df

,Наименование,Цена,Количество отзывов,Рейтинг,Цвет,Категория,Материал
0,джинсовая куртка zara лето-2021 свободного пок...,5166,61,4.3,камень,куртка,джинс
1,рубашка из поплина с оборками из коллекции zw ...,4671,32,3.5,белый,рубашка,поплин
2,мягкое трикотажное платье миди бутылочно-зеленый,6246,193,2.9,зеленый,платье,трикотаж
3,широкие джинсы с завышенной талией из коллекци...,3123,276,3.2,черный,джинсы,джинс
4,джинсы-карго с парашютом средне-синий,3015,147,3.4,синий,джинсы,джинс
...,...,...,...,...,...,...,...
988,джинсовая юбка-миди с накидкой z1975 белый,5166,288,1.2,белый,юбка,джинс
989,брюки из хлопка и льна устрично-белый,8037,60,3.0,"белый, устрично-белый",брюки,"хлоп, льн"
990,текстурированное драпированное хлопчатобумажно...,8757,127,4.4,бежевый,платье,хлоп
991,верхняя рубашка из искусственной кожи зеленый,12348,133,1.9,зеленый,рубашка,кож


In [6]:
# Создание нового столбца "Метка"
df['Метка'] = ''

# Добавление меток в столбец "Метка" на основе сравнения значений в столбце "Рейтинг" с числом 4
df['Метка'] = df['Рейтинг'].apply(lambda x: 1 if x > 3 else (0 if x == 3 else 0))

# Сохранение изменений в новый Excel-файл
df.to_excel('Данные_обработанные_ЖО 1 мод.xlsx', index=False)

In [7]:
# Создание нового столбца "Метка"
df['Метка'] = ''

# Добавление меток в столбец "Метка" на основе сравнения значений в столбце "Рейтинг" с числом 4
df['Метка'] = df['Рейтинг'].apply(lambda x: 1 if x > 4 else (0 if x == 4 else 0))

# Сохранение изменений в новый Excel-файл
df.to_excel('Данные_обработанные_ЖО 2 мод.xlsx', index=False)


In [8]:
# Создание нового столбца "Метка"
df['Метка'] = ''

# Добавление меток в столбец "Метка" на основе сравнения значений в столбце "Рейтинг" с числом 4
df['Метка'] = df['Рейтинг'].apply(lambda x: 1 if x == 5 else 0)

# Сохранение изменений в новый Excel-файл
df.to_excel('Данные_обработанные_ЖО 3 мод.xlsx', index=False)

In [104]:
# Преобразование столбцов в категориальные
#df['Цвет'] = df['Цвет'].astype('category')
#df['Категория'] = df['Категория'].astype('category')
#df['Материал'] = df['Материал'].astype('category')
# Сохранение изменений в новый Excel-файл
#df.to_excel('Данные_обработанные_ЖО.xlsx', index=False)  # index=False для удаления индексов строк
# Теперь можно безопасно использовать.cat.accessor без ошибок
#print(df['Цвет'].cat.categories)
#print(df['Категория'].cat.categories)
#print(df['Материал'].cat.categories)

# Преобразование категориальных переменных в числа
#df['Цвет'] = df['Цвет'].astype('category').cat.codes
#df['Категория'] = df['Категория'].astype('category').cat.codes
#df['Материал'] = df['Материал'].astype('category').cat.codes
# Сохранение изменений в новый Excel-файл
#df.to_excel('Данные_обработанные_ЖО.xlsx', index=False)

##РАЗДЕЛЕНИЕ НА ТЕСТОВЫЕ И ОБУЧАЮЩИЕ ВЫБОРКИ

In [9]:
import pandas as pd

# Путь к вашему файлу
file_path1 = '/content/Данные_обработанные_ЖО 1 мод.xlsx'
file_path2 = '/content/Данные_обработанные_ЖО 2 мод.xlsx'
file_path3 = '/content/Данные_обработанные_ЖО 3 мод.xlsx'

# Загрузка данных
df1 = pd.read_excel(file_path1)
df2 = pd.read_excel(file_path2)
df3 = pd.read_excel(file_path3)

# Выбор соотношения для разделения
ratio1 = 0.8
ratio2 = 0.7

# Вычисление количества строк для каждой выборки соотношением (0.8)
train_size1 = int(len(df1) * ratio1)
test_size1 = len(df1) - train_size1

train_size2 = int(len(df2) * ratio1)
test_size2 = len(df2) - train_size2

train_size3 = int(len(df3) * ratio1)
test_size3 = len(df3) - train_size3

# Разделение данных
train_df1 = df.iloc[:train_size1]
test_df1 = df.iloc[train_size1]

train_df2 = df.iloc[:train_size2]
test_df2 = df.iloc[train_size2]

train_df3 = df.iloc[:train_size3]
test_df3 = df.iloc[train_size3]


# Сохранение обучающей выборки
train_file_path1 = '/content/Обучающая_выборка для 1 м.xlsx'
train_df1.to_excel(train_file_path1, index=False)

train_file_path2 = '/content/Обучающая_выборка для 2 м.xlsx'
train_df2.to_excel(train_file_path2, index=False)

train_file_path3 = '/content/Обучающая_выборка для 3 м.xlsx'
train_df3.to_excel(train_file_path3, index=False)


In [10]:
df = pd.read_excel('/content/Обучающая_выборка для 1 м.xlsx')
df

,Наименование,Цена,Количество отзывов,Рейтинг,Цвет,Категория,Материал,Метка
0,джинсовая куртка zara лето-2021 свободного пок...,5166,61,4.3,камень,куртка,джинс,0
1,рубашка из поплина с оборками из коллекции zw ...,4671,32,3.5,белый,рубашка,поплин,0
2,мягкое трикотажное платье миди бутылочно-зеленый,6246,193,2.9,зеленый,платье,трикотаж,0
3,широкие джинсы с завышенной талией из коллекци...,3123,276,3.2,черный,джинсы,джинс,0
4,джинсы-карго с парашютом средне-синий,3015,147,3.4,синий,джинсы,джинс,0
...,...,...,...,...,...,...,...,...
789,волнистые жаккардовые брюки верблюд,5166,235,3.4,верблюд,брюки,жаккард,0
790,футболка в рубчик синий,2295,126,1.5,синий,футболка,рубч,0
791,футболка в рубчик на пуговицах - лимитированна...,3375,139,4.7,белый,футболка,рубч,0
792,блейзер для костюма из смеси шерсти черный,17010,99,1.8,черный,блейзер,шерст,0


In [11]:
from sklearn.preprocessing import LabelEncoder

# Преобразование категориальных данных в числовые индексы
encoder = LabelEncoder()
df['Цвет'] = encoder.fit_transform(df['Цвет'])
df['Материал'] = encoder.fit_transform(df['Материал'])
df['Категория'] = encoder.fit_transform(df['Категория'])

# Сохранение изменений обратно в Excel файл
df.to_excel('/content/Обучающая_выборка для 1 м.xlsx', index=False)

In [12]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Загрузка данных из Excel файла
df = pd.read_excel('/content/Обучающая_выборка для 1 м.xlsx')
# Создание объекта MinMaxScaler
scaler = MinMaxScaler()

# Применение масштабирования к выбранным столбцам
df[['Цвет', 'Материал', 'Цена', 'Категория']] = scaler.fit_transform(df[['Цвет', 'Материал', 'Цена', 'Категория']])

# Вывод измененного DataFrame
df
# Сохранение изменений в новый Excel-файл
df.to_excel('/content/Обучающая_выборка для 1 м.xlsx', index=False)


##Построение модели

In [13]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, KFold, StratifiedShuffleSplit
from sklearn.preprocessing import RobustScaler, Normalizer


In [14]:
# Удаление столбца по названию
column_to_delete_name = 'Наименование'
df.drop(columns=[column_to_delete_name], inplace=True)
# Сохранение изменений в новый Excel-файл
df.to_excel('/content/Обучающая_выборка для 1 м.xlsx', index=False)  # index=False для удаления индексов строк

In [15]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_ЖО 1 мод.xlsx')

# Создание объекта LabelEncoder
le = LabelEncoder()

# Применение LabelEncoder к категориальным столбцам
for column in ["Цвет", "Материал", "Категория"]:
    data[column] = le.fit_transform(data[column])

# Разделение на признаки и целевую переменную
X = data[["Цвет", "Материал", "Категория", "Цена"]]
y = data["Метка"]

# Добавление константы (если требуется) к признакам
#X = sm.add_constant(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# Обучение модели логистической регрессии на обучающей выборке
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()
print("для соотношение 20/80:")
print(result.summary())

# Повторное разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

# Обучение модели логистической регрессии на обучающей выборке
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()
print("для соотношение 30/70:")
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.687456
         Iterations 4
для соотношение 20/80:
                           Logit Regression Results                           
Dep. Variable:                  Метка   No. Observations:                  198
Model:                          Logit   Df Residuals:                      194
Method:                           MLE   Df Model:                            3
Date:                Wed, 29 May 2024   Pseudo R-squ.:                0.008210
Time:                        14:31:33   Log-Likelihood:                -136.12
converged:                       True   LL-Null:                       -137.24
Covariance Type:            nonrobust   LLR p-value:                    0.5215
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Цвет           0.0118      0.011      1.088      0.277      -0.009       0.033
Матери

In [16]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_ЖО 2 мод.xlsx')

# Создание объекта LabelEncoder
le = LabelEncoder()

# Применение LabelEncoder к категориальным столбцам
for column in ["Цвет", "Материал", "Категория"]:
    data[column] = le.fit_transform(data[column])

# Разделение на признаки и целевую переменную
X = data[["Цвет", "Материал", "Категория", "Цена"]]
y = data["Метка"]

# Добавление константы (если требуется) к признакам
#X = sm.add_constant(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# Обучение модели логистической регрессии на обучающей выборке
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()
print("Соотношение 20/80:")
print(result.summary())

# Повторное разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

# Обучение модели логистической регрессии на обучающей выборке
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()
print("Соотношение 30/70:")
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.582598
         Iterations 5
Соотношение 20/80:
                           Logit Regression Results                           
Dep. Variable:                  Метка   No. Observations:                  198
Model:                          Logit   Df Residuals:                      194
Method:                           MLE   Df Model:                            3
Date:                Wed, 29 May 2024   Pseudo R-squ.:                -0.03098
Time:                        14:31:38   Log-Likelihood:                -115.35
converged:                       True   LL-Null:                       -111.89
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Цвет          -0.0179      0.012     -1.464      0.143      -0.042       0.006
Материал  

In [17]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_ЖО 2 мод.xlsx')

# Создание объекта LabelEncoder
le = LabelEncoder()

# Применение LabelEncoder к категориальным столбцам
for column in ["Цвет", "Материал", "Категория"]:
    data[column] = le.fit_transform(data[column])

# Разделение на признаки и целевую переменную
X = data[["Цвет", "Материал", "Категория", "Цена"]]
y = data["Метка"]

# Добавление константы (если требуется) к признакам
#X = sm.add_constant(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# Обучение модели логистической регрессии на обучающей выборке
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()
print("Соотношение 20/80:")
print(result.summary())

# Повторное разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

# Обучение модели логистической регрессии на обучающей выборке
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()
print("Соотношение 30/70:")
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.582598
         Iterations 5
Соотношение 20/80:
                           Logit Regression Results                           
Dep. Variable:                  Метка   No. Observations:                  198
Model:                          Logit   Df Residuals:                      194
Method:                           MLE   Df Model:                            3
Date:                Wed, 29 May 2024   Pseudo R-squ.:                -0.03098
Time:                        14:31:43   Log-Likelihood:                -115.35
converged:                       True   LL-Null:                       -111.89
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Цвет          -0.0179      0.012     -1.464      0.143      -0.042       0.006
Материал  

In [66]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_ЖО 3 мод.xlsx')

# Разделение на признаки и целевую переменную
X = data[["Цвет", "Материал", "Категория", "Цена"]]
y = data["Метка"]

# Добавление константы (если требуется) к признакам
#X = sm.add_constant(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# Обучение модели логистической регрессии на обучающей выборке
logit_model = sm.Logit(y, X)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.073304
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                  Метка   No. Observations:                  993
Model:                          Logit   Df Residuals:                      989
Method:                           MLE   Df Model:                            3
Date:                Wed, 29 May 2024   Pseudo R-squ.:                 -0.1213
Time:                        05:11:35   Log-Likelihood:                -72.791
converged:                       True   LL-Null:                       -64.917
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Цвет          -1.3767      0.597     -2.305      0.021      -2.548      -0.206
Материал      -6.5987      1

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_ЖО.xlsx')

# Разделение на признаки и целевую переменную
X = data[["Цвет", "Материал", "Категория", "Цена"]]
y = data["Метка"]

# Добавление константы (если требуется) к признакам
X = sm.add_constant(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# Обучение модели логистической регрессии на обучающей выборке
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()
print("Обучающая выборка:")
print(result.summary())

# Тестирование модели на тестовой выборке
predictions = result.predict(sm.add_constant(X_test))
print("\nТестовая выборка:")
print(predictions)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_ЖО.xlsx')

# Подготовка данных
categorical_features = ['Материал', 'Категория', 'Цвет', 'Цена']
data_encoded = pd.get_dummies(data, columns=categorical_features)

# Разделение данных на признаки и целевую переменную
X = data_encoded.drop('Метка', axis=1)
y = data_encoded['Метка']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

# Создание и обучение модели Naive Bayes
model_nb = GaussianNB()
model_nb.fit(X_train, y_train)

# Предсказание на тестовой выборке
predictions_nb = model_nb.predict(X_test)

# Оценка модели Naive Bayes на тренировочной выборке
accuracy_nb_train = accuracy_score(y_train, model_nb.predict(X_train))
print(f'Train Accuracy with Naive Bayes: {accuracy_nb_train}')

# Оценка модели Naive Bayes на тестовой выборке
accuracy_nb_test = accuracy_score(y_test, predictions_nb)
print(f'Test Accuracy with Naive Bayes: {accuracy_nb_test}')


Train Accuracy with Naive Bayes: 0.49622166246851385
Test Accuracy with Naive Bayes: 0.39932885906040266


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier  # Изменено от KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_ЖО.xlsx')

# Подготовка данных
categorical_features = ['Материал', 'Категория', 'Цвет', 'Цена']
data_encoded = pd.get_dummies(data, columns=categorical_features)

# Разделение данных на признаки и целевую переменную
X = data_encoded.drop('Метка', axis=1)
y = data_encoded['Метка']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

# Создание и обучение модели дерева решений
model_tree = DecisionTreeClassifier(max_depth=10)  # Пример параметра max_depth
model_tree.fit(X_train, y_train)

# Предсказание на тестовой выборке
predictions_tree = model_tree.predict(X_test)

# Оценка модели дерева решений на тренировочной выборке
accuracy_tree_train = accuracy_score(y_train, model_tree.predict(X_train))
print(f'Train Accuracy with Tree: {accuracy_tree_train}')

# Оценка модели дерева решений на тестовой выборке
accuracy_tree_test = accuracy_score(y_test, predictions_tree)
print(f'Test Accuracy with Tree: {accuracy_tree_test}')

Train Accuracy with Tree: 1.0
Test Accuracy with Tree: 1.0


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC # Импорт классификатора SVM
from sklearn.metrics import accuracy_score

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_ЖО.xlsx')

# Подготовка данных
categorical_features = ['Материал', 'Категория', 'Цвет', 'Цена']
data_encoded = pd.get_dummies(data, columns=categorical_features)

# Разделение данных на признаки и целевую переменную
X = data_encoded.drop('Метка', axis=1)
y = data_encoded['Метка']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

# Создание и обучение модели SVM
model_svm = SVC(kernel='linear') # Вы можете выбрать другой ядро, если необходимо
model_svm.fit(X_train, y_train)

# Предсказание на тестовой выборке
predictions_svm = model_svm.predict(X_test)

# Оценка модели SVM на тренировочной выборке
accuracy_svm_train = accuracy_score(y_train, model_svm.predict(X_train))
print(f'Train Accuracy with SVM: {accuracy_svm_train}')

# Оценка модели SVM на тестовой выборке
accuracy_svm_test = accuracy_score(y_test, predictions_svm)
print(f'Test Accuracy with SVM: {accuracy_svm_test}')


Train Accuracy with SVM: 0.9924433249370277
Test Accuracy with SVM: 0.9865771812080537


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_ЖО.xlsx')
# Подготовка данных
categorical_features = ['Материал', 'Категория', 'Цвет', 'Цена']
data_encoded = pd.get_dummies(data, columns=categorical_features)
# Разделение данных на признаки и целевую переменную
X = data_encoded.drop('Метка', axis=1)
y = data_encoded['Метка']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

# Создание и обучение модели Random Forest
model_rf = RandomForestClassifier(n_estimators=100)  # Используйте подходящее количество деревьев
model_rf.fit(X_train, y_train)

# Предсказание на тестовой выборке
predictions_rf = model_rf.predict(X_test)

# Оценка модели Random Forest на тренировочной выборке
accuracy_rf_train = accuracy_score(y_train, model_rf.predict(X_train))
print(f'Train Accuracy with Random Forest: {accuracy_rf_train}')

# Оценка модели Random Forest на тестовой выборке
accuracy_rf_test = accuracy_score(y_test, predictions_rf)
print(f'Test Accuracy with Random Forest: {accuracy_rf_test}')


Train Accuracy with Random Forest: 1.0
Test Accuracy with Random Forest: 1.0


In [ ]:
def predict_success(user_input):
    # Предполагаем, что user_input - это словарь с ключами соответствующих столбцам в X
    # Например: {'Материал': 'стекло', 'Категория': 'косметика',...}

    # Подготовка данных для предсказания
    # Здесь предполагается, что все необходимые преобразования уже выполнены
    # Например, кодирование категориальных переменных
    encoded_input = encode_user_input(user_input)

    # Исправление: Передаем encoded_input как двумерный массив
    prediction = model_tree.predict(encoded_input.reshape(1, -1))

    return prediction[0]

def encode_user_input(input_dict):
    # Функция для кодирования входных данных пользователя
    # Здесь должна быть логика преобразования категориальных переменных в числовые
    # и других необходимых преобразований
    pass

# Получение ввода от пользователя
user_data = {
    'Материал': input("Введите материал: "),
    'Категория': input("Введите категорию: "),
    'Цвет': input("Введите цвет: "),
    'Цена': input("Введите цену: "),
}

# Предсказание успешности запуска
prediction = predict_success(user_data)
if prediction == 1:
    print("Запуск ожидается успешным.")
else:
    print("Запуск не гарантирован.")

Введите материал: 0.125000
Введите категорию: 0.52179
Введите цвет: 0.3
Введите цену: 0.12


AttributeError: 'NoneType' object has no attribute 'reshape'

In [ ]:
import numpy as np

def predict_success(user_input):
    # Предполагаем, что user_input - это словарь с ключами соответствующих столбцам в X
    # Например: {'Материал': 'стекло', 'Категория': 'косметика',...}

    # Подготовка данных для предсказания
    # Здесь предполагается, что все необходимые преобразования уже выполнены
    # Например, кодирование категориальных переменных
    encoded_input = encode_user_input(user_input)

    # Проверяем, что encoded_input не None и имеет нужную форму
    if encoded_input is not None:
        prediction = model_tree.predict(encoded_input.reshape(1, -1))
    else:
        raise ValueError("Encoded input is None")

    return prediction[0]

def encode_user_input(input_dict):
    # Пример реализации функции для кодирования входных данных пользователя
    # Здесь должна быть логика преобразования категориальных переменных в числовые
    # и других необходимых преобразований

    # Для демонстрации создадим простой массив из словаря
    # В реальном случае здесь должен быть ваш код преобразования
    return np.array(list(input_dict.values()))

# Получение ввода от пользователя
user_data = {
    'Материал': input("Введите материал: "),
    'Категория': input("Введите категорию: "),
    'Цвет': input("Введите цвет: "),
    'Цена': input("Введите цену: "),
}

# Предсказание успешности запуска
try:
    prediction = predict_success(user_data)
    if prediction == 1:
        print("Запуск ожидается успешным.")
    else:
        print("Запуск не гарантирован.")
except Exception as e:
    print(f"Произошла ошибка: {e}")


Введите материал: 1
Введите категорию: 2
Введите цвет: 3
Введите цену: 0.2232
Произошла ошибка: X has 4 features, but DecisionTreeClassifier is expecting 147 features as input.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


В этом коде мы добавили шаг кросс-валидации перед обучением модели на всей обучающей выборке. Кросс-валидация выполняется с помощью параметра cv=5, что означает, что данные будут разделены на 5 частей, и модель будет обучаться на 4 частях, а затем тестируеться на оставшейся части. Этот процесс повторяется 5 раз, каждый раз с другой частью данных в качестве тестового набора. Результаты кросс-валидации (scores) показывают, как модель работает на различных подмножествах данных, что может помочь выявить признаки переобучения (например, если разброс оценок слишком велик). Кросс-валидация позволяет оценить стабильность модели, так как она выполняется несколько раз с различными разделениями данных. Если результаты близки друг к другу, это говорит о том, что модель не слишком чувствительна к выбору конкретных данных для обучения и тестирования, что является положительным признаком

In [ ]:
import numpy as np

def predict_success(user_input):
    # Подготовка данных для предсказания
    encoded_input = encode_user_input(user_input)

    # Проверяем, что encoded_input не None и имеет нужную форму
    if encoded_input is not None:
        prediction = model_tree.predict(encoded_input.reshape(1, -1))
    else:
        raise ValueError("Encoded input is None")

    return prediction[0]

def encode_user_input(input_dict):
    # Создаем базовый массив из словаря, предполагая, что все значения уже числовые
    numeric_values = [float(value) for value in input_dict.values()]

    # Заменяем отсутствующие значения на 0, чтобы обеспечить полное соответствие размеру модели
    expected_features = 147  # Значение, основанное на ошибке "X has 4 features, but DecisionTreeClassifier is expecting 147 features"
    if len(numeric_values) < expected_features:
        numeric_values.extend([0] * (expected_features - len(numeric_values)))

    # Преобразуем список значений в массив NumPy с нужной формой
    return np.array(numeric_values).reshape(1, -1)

# Получение ввода от пользователя
user_data = {
    'Материал': input("Введите материал: "),
    'Категория': input("Введите категорию: "),
    'Цвет': input("Введите цвет: "),
    'Цена': input("Введите цену: "),
}

# Предсказание успешности запуска
try:
    prediction = predict_success(user_data)
    if prediction == 1:
        print("Запуск ожидается успешным.")
    else:
        print("Запуск не ожидается успешным.")
except Exception as e:
    print(f"Произошла ошибка: {e}")

Введите материал: 3
Введите категорию: 4
Введите цвет: 5
Введите цену: 0.231134
Запуск ожидается успешным.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [ ]:
import numpy as np

def predict_success(user_input):
    # Подготовка данных для предсказания
    encoded_input = encode_user_input(user_input)

    # Проверяем, что encoded_input не None и имеет нужную форму
    if encoded_input is not None:
        prediction = model_tree.predict(encoded_input.reshape(1, -1))
    else:
        raise ValueError("Encoded input is None")

    return prediction[0]

def encode_user_input(input_dict):
    # Создаем базовый массив из словаря, предполагая, что все значения уже числовые
    numeric_values = [float(value) for value in input_dict.values()]

    # Заменяем отсутствующие значения на 0, чтобы обеспечить полное соответствие размеру модели
    expected_features = 147  # Значение, основанное на ошибке "X has 4 features, but DecisionTreeClassifier is expecting 147 features"
    if len(numeric_values) < expected_features:
        numeric_values.extend([0] * (expected_features - len(numeric_values)))

    # Преобразуем список значений в массив NumPy с нужной формой
    return np.array(numeric_values).reshape(1, -1)

# Получение ввода от пользователя
user_data = {
    'Материал': input("Введите материал: "),
    'Категория': input("Введите категорию: "),
    'Цвет': input("Введите цвет: "),
    'Цена': input("Введите цену: "),
}

# Предсказание успешности запуска
try:
    prediction = predict_success(user_data)
    if prediction == 1:
        print("Запуск ожидается успешным.")
    else:
        print("Запуск не ожидается успешным.")
except Exception as e:
    print(f"Произошла ошибка: {e}")

Введите материал: 0.833333
Введите категорию: 0.563933
Введите цвет: 0.233333
Введите цену: 0.065378
Запуск не ожидается успешным.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Регуляризационная логистическая регрессия

####МУЖЧИНЫ

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Чтение данных из Excel-файла
df = pd.read_excel('/content/Ассортимент брендов_МО.xlsx')

# Поиск минимального и максимального значений в столбце "Количество отзывов"
min_reviews = df['Количество отзывов'].min()
max_reviews = df['Количество отзывов'].max()

# Печать результатов
print(f"Минимальное количество отзывов: {min_reviews}")
print(f"Максимальное количество отзывов: {max_reviews}")

Минимальное количество отзывов: 3
Максимальное количество отзывов: 190


In [ ]:
# Удаление строк, где количество отзывов меньше 20
df = df[df['Количество отзывов'] >= 20]
# Перевод значений в столбце "Наименование товара" в нижний регистр и удаление пунктуации
df['Наименование'] = df['Наименование'].str.lower().str.replace('[^\w\s]', '')
# Удаление символа рубля из столбца "Цена"
df['Цена'] = df['Цена'].str.replace('₽', '')
# Удаление пробелов внутри чисел в столбце "Цена"
df['Цена'] = df['Цена'].str.replace('\s+', '', regex=True)
# Удаление всех чисел после точки в столбце "Цена"
df['Цена'] = df['Цена'].str.replace('\.\d+', '', regex=True)
# Сохранение изменений в новый Excel-файл
df.to_excel('Данные_обработанные_МО.xlsx', index=False)  # index=False для удаления индексов строк

<ipython-input-32-0b2739e34eeb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Наименование'] = df['Наименование'].str.lower().str.replace('[^\w\s]', '')
<ipython-input-32-0b2739e34eeb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Цена'] = df['Цена'].str.replace('₽', '')
<ipython-input-32-0b2739e34eeb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [ ]:
df

,Наименование,Цена,Средний рейтинг,Количество отзывов
0,брюки из текстурированного хлопка-чино синий,6831,3.6,48
1,брюки-чиносы с прямыми штанинами бежевый,5076,4.1,87
2,рваные джинсовые шорты-бермуды синий,5076,1.4,106
3,джинсы с прямыми штанинами хаки,5076,2.0,73
4,футболка 5s slim fit белый/черный,6237,5.0,43
...,...,...,...,...
2298,облегающий топ с капюшоном и принтом большого ...,5130,1.2,162
2300,расслабленная посадка с принтом и капюшоном ко...,3960,3.2,81
2301,расслабленная посадка с принтом и капюшоном зе...,3960,3.3,90
2302,облегающий топ с капюшоном и принтом большого ...,4176,3.8,177


In [ ]:
# Функция для поиска цветов в строке
def find_colors(row):
    # Чтение списка цветов из файла
    colors = pd.read_csv('/content/Цвета.txt', header=None)[0].tolist()
    return ', '.join(color for color in colors if color in row['Наименование'])
# Применяем функцию к каждой строке и сохраняем результаты в новый столбец "Цвет"
df['Цвет'] = df.apply(find_colors, axis=1)

# Подсчет общего количества NaN в столбце "Цвет"
num_missing_colors = df['Цвет'].isnull().sum()
# Печать общего количества NaN в столбце "Цвет"


def find_category(row):
  # Чтение списка цветов из файла
    category = pd.read_csv('/content/Категории.txt', header=None)[0].tolist()
    return ', '.join(category for category in category if category in row['Наименование'])
# Применяем функцию к каждой строке и сохраняем результаты в новый столбец "Цвет"
df['Категория'] = df.apply(find_category, axis=1)

def find_material(row):
  # Чтение списка цветов из файла
    material = pd.read_csv('/content/Материал.txt', header=None)[0].tolist()
    return ', '.join(material for material in material if material in row['Наименование'])
# Применяем функцию к каждой строке и сохраняем результаты в новый столбец "Цвет"
df['Материал'] = df.apply(find_material, axis=1)

# Преобразование пустых строк в NaN
df['Цвет'] = df['Цвет'].replace('', np.nan)
df['Категория'] = df['Категория'].replace('', np.nan)
df['Материал'] = df['Материал'].replace('', np.nan)

# Сохранение изменений в новый Excel-файл
df.to_excel('Данные_обработанные_МО.xlsx', index=False)  # index=False для удаления индексов строк

# Подсчет общего количества NaN в столбце "Цвет"
num_missing_colors = df['Цвет'].isnull().sum()
num_missing_category = df['Категория'].isnull().sum()
num_missing_material = df['Материал'].isnull().sum()

print(f"Количество NaN в столбце 'Цвет':{num_missing_colors}")
print(f"Количество NaN в столбце 'Категория':{num_missing_category}")
print(f"Количество NaN в столбце 'Материал':{num_missing_material}")

Количество NaN в столбце 'Цвет':341
Количество NaN в столбце 'Категория':553
Количество NaN в столбце 'Материал':1454


In [ ]:
# Удаление строк с пропусками в указанных столбцах
df = df.dropna(subset=['Материал', 'Категория', 'Цвет', 'Цена'])

# Сохранение изменений в новый Excel-файл
df.to_excel('Данные_обработанные_МО.xlsx', index=False)  # index=False для удаления индексов строк

In [ ]:
df = pd.read_excel('/content/Данные_обработанные_МО.xlsx')
df

,Наименование,Цена,Средний рейтинг,Количество отзывов,Цвет,Категория,Материал,Метка
0,брюки из текстурированного хлопка-чино синий,6831,3.6,48,синий,брюки,хлоп,1
1,рваные джинсовые шорты-бермуды синий,5076,1.4,106,синий,"шорты, шорты-бермуды",джинс,0
2,джинсы с прямыми штанинами хаки,5076,2.0,73,хаки,джинсы,джинс,0
3,облегающие джинсы синий,4581,3.8,69,синий,джинсы,джинс,1
4,облегающая футболка-поло из смесового шелка те...,6237,3.4,35,серый,футболка,шелк,0
...,...,...,...,...,...,...,...,...
537,выстиранные джинсы с принтом светло-бежевый,8586,3.8,189,"бежевый, светло-бежевый",джинсы,джинс,1
538,трикотажная футболка с абстрактным принтом черный,4014,1.6,190,черный,футболка,трикотаж,0
539,трикотажная футболка с принтом в виде спрея серый,4014,4.4,144,серый,футболка,трикотаж,1
540,облегающий трикотажный топ с принтом большого ...,3591,1.0,65,черный,топ,трикотаж,0


In [ ]:
# Создание нового столбца "Метка"
df['Метка'] = ''

# Добавление меток в столбец "Метка" на основе сравнения значений в столбце "Рейтинг" с числом 4
df['Метка'] = df['Средний рейтинг'].apply(lambda x: 1 if x > 3.5 else (0 if x == 3.5 else 0))

# Сохранение изменений в новый Excel-файл
df.to_excel('Данные_обработанные_МО.xlsx', index=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Преобразование категориальных данных в числовые индексы
encoder = LabelEncoder()
df['Цвет'] = encoder.fit_transform(df['Цвет'])
df['Материал'] = encoder.fit_transform(df['Материал'])
df['Категория'] = encoder.fit_transform(df['Категория'])

# Сохранение изменений обратно в Excel файл
df.to_excel('/content/Данные_обработанные_МО.xlsx', index=False)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Загрузка данных из Excel файла
df = pd.read_excel('Данные_обработанные_МО.xlsx')
# Создание объекта MinMaxScaler
scaler = MinMaxScaler()

# Применение масштабирования к выбранным столбцам
df[['Цвет', 'Материал', 'Цена', 'Категория']] = scaler.fit_transform(df[['Цвет', 'Материал', 'Цена', 'Категория']])
df.to_excel('/content/Данные_обработанные_ЖО.xlsx', index=False)
# Вывод измененного DataFrame
df

,Наименование,Цена,Средний рейтинг,Количество отзывов,Цвет,Категория,Материал,Метка
0,брюки из текстурированного хлопка-чино синий,0.142659,3.6,48,0.684211,0.071429,0.852941,1
1,рваные джинсовые шорты-бермуды синий,0.097572,1.4,106,0.684211,1.000000,0.147059,0
2,джинсы с прямыми штанинами хаки,0.097572,2.0,73,0.921053,0.142857,0.147059,0
3,облегающие джинсы синий,0.084855,3.8,69,0.684211,0.142857,0.147059,1
4,облегающая футболка-поло из смесового шелка те...,0.127399,3.4,35,0.631579,0.857143,0.941176,0
...,...,...,...,...,...,...,...,...
537,выстиранные джинсы с принтом светло-бежевый,0.187746,3.8,189,0.078947,0.142857,0.147059,1
538,трикотажная футболка с абстрактным принтом черный,0.070289,1.6,190,0.947368,0.857143,0.500000,0
539,трикотажная футболка с принтом в виде спрея серый,0.070289,4.4,144,0.631579,0.857143,0.500000,1
540,облегающий трикотажный топ с принтом большого ...,0.059422,1.0,65,0.947368,0.785714,0.500000,0


##Построение моделей


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, KFold, StratifiedShuffleSplit
from sklearn.preprocessing import RobustScaler, Normalizer


In [ ]:
# Удаление столбца по названию
column_to_delete_name = 'Наименование'
df.drop(columns=[column_to_delete_name], inplace=True)
# Сохранение изменений в новый Excel-файл
df.to_excel('/content/Данные_обработанные_МО.xlsx', index=False)  # index=False для удаления индексов строк

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_МО.xlsx')

# Подготовка данных
categorical_features = ['Материал', 'Категория', 'Цвет', 'Цена']
data_encoded = pd.get_dummies(data, columns=categorical_features)

# Разделение данных на признаки и целевую переменную
X = data_encoded.drop('Метка', axis=1)
y = data_encoded['Метка']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

# Создание и обучение модели KNN
model_knn = KNeighborsClassifier(n_neighbors=10)
model_knn.fit(X_train, y_train)

# Предсказание на тестовой выборке
predictions_knn = model_knn.predict(X_test)

# Оценка модели KNN на тренировочной выборке
accuracy_knn_train = accuracy_score(y_train, model_knn.predict(X_train))
print(f'Train Accuracy with KNN: {accuracy_knn_train}')

# Оценка модели KNN на тестовой выборке
accuracy_knn_test = accuracy_score(y_test, predictions_knn)
print(f'Test Accuracy with KNN: {accuracy_knn_test}')

Train Accuracy with KNN: 0.7037037037037037
Test Accuracy with KNN: 0.6993865030674846


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_МО.xlsx')

# Подготовка данных
categorical_features = ['Материал', 'Категория', 'Цвет', 'Цена']
data_encoded = pd.get_dummies(data, columns=categorical_features)

# Разделение данных на признаки и целевую переменную
X = data_encoded.drop('Метка', axis=1)
y = data_encoded['Метка']

# Нормализация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.5, random_state=42)

# Создание и обучение модели логистической регрессии
model_lr = LogisticRegression(max_iter=1000) # Увеличение итераций для стабильности обучения
model_lr.fit(X_train, y_train)

# Предсказание на тестовой выборке
predictions_lr = model_lr.predict(X_test)

# Оценка модели логистической регрессии на тренировочной выборке
accuracy_lr_train = accuracy_score(y_train, model_lr.predict(X_train))
r_squared_train = model_lr.score(X_train, y_train)
print(f'Train Accuracy with LR: {accuracy_lr_train}, R-Squared Train: {r_squared_train}')

# Оценка модели логистической регрессии на тестовой выборке
accuracy_lr_test = accuracy_score(y_test, predictions_lr)
r_squared_test = model_lr.score(X_test, y_test)
print(f'Test Accuracy with LR: {accuracy_lr_test}, R-Squared Test: {r_squared_test}')


Train Accuracy with LR: 0.996309963099631, R-Squared Train: 0.996309963099631
Test Accuracy with LR: 0.9261992619926199, R-Squared Test: 0.9261992619926199


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_МО.xlsx')

# Подготовка данных
categorical_features = ['Материал', 'Категория', 'Цвет', 'Цена']
data_encoded = pd.get_dummies(data, columns=categorical_features)

# Разделение данных на признаки и целевую переменную
X = data_encoded.drop('Метка', axis=1)
y = data_encoded['Метка']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

# Создание и обучение модели Naive Bayes
model_nb = GaussianNB()
model_nb.fit(X_train, y_train)

# Предсказание на тестовой выборке
predictions_nb = model_nb.predict(X_test)

# Оценка модели Naive Bayes на тренировочной выборке
accuracy_nb_train = accuracy_score(y_train, model_nb.predict(X_train))
print(f'Train Accuracy with Naive Bayes: {accuracy_nb_train}')

# Оценка модели Naive Bayes на тестовой выборке
accuracy_nb_test = accuracy_score(y_test, predictions_nb)
print(f'Test Accuracy with Naive Bayes: {accuracy_nb_test}')

Train Accuracy with Naive Bayes: 0.5601851851851852
Test Accuracy with Naive Bayes: 0.3803680981595092


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier  # Изменено от KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Загрузка данных
data = pd.read_excel('/content/Данные_обработанные_МО.xlsx')

# Подготовка данных
categorical_features = ['Материал', 'Категория', 'Цвет', 'Цена']
data_encoded = pd.get_dummies(data, columns=categorical_features)

# Разделение данных на признаки и целевую переменную
X = data_encoded.drop('Метка', axis=1)
y = data_encoded['Метка']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

# Создание и обучение модели дерева решений
model_tree = DecisionTreeClassifier(max_depth=10)  # Пример параметра max_depth
model_tree.fit(X_train, y_train)

# Предсказание на тестовой выборке
predictions_tree = model_tree.predict(X_test)

# Оценка модели дерева решений на тренировочной выборке
accuracy_tree_train = accuracy_score(y_train, model_tree.predict(X_train))
print(f'Train Accuracy with Tree: {accuracy_tree_train}')

# Оценка модели дерева решений на тестовой выборке
accuracy_tree_test = accuracy_score(y_test, predictions_tree)
print(f'Test Accuracy with Tree: {accuracy_tree_test}')

Train Accuracy with Tree: 1.0
Test Accuracy with Tree: 1.0
